# PLAYGROUND for AgentGENius

In [ ]:
import logfire
import nest_asyncio
from dotenv import load_dotenv
from rich import print

from agentgenius import AgentDef, AgentParams, Task, TaskDef
from agentgenius.builtin_tools import get_datetime, get_location_by_ip, get_user_ip
from agentgenius.tasks import Task, TaskDef
from agentgenius.tools import ToolDef, ToolSet
from agentgenius.utils import TypeAdapterMixin

nest_asyncio.apply()
load_dotenv()

In [ ]:
agent_def = AgentDef(
    model="openai:gpt-4o-mini",
    name="assistant",
    system_prompt="You are a helpful assistant.",
    params=AgentParams(
        result_type=TaskDef,
        retries=2,
    ),
)

# json_data = agent_def.model_dump_json()
# print(json_data)
# agent_def = AgentDef.model_validate_json(json_data)
# print(agent_def)
# print(agent_def.model_json_schema())
print(agent_def.params.__dict__)

In [ ]:
def foo():
    return "bar"


tool = ToolDef("foo")

json_data = tool.model_dump_json()
print(json_data)
tool = ToolDef.model_validate_json(json_data)
print(tool)

In [ ]:
toolset = ToolSet(["foo"])
json_data = toolset.model_dump_json()
print(json_data)
toolset = ToolSet.model_validate_json(json_data)
print(toolset)

In [ ]:
task_def = TaskDef(
    name="get time",
    question="what time is it?",
    priority=1,
    toolset=ToolSet(
        [get_datetime, foo],
    ),
)

json_data = task_def.model_dump_json(exclude_unset=True, exclude_defaults=True)
print(json_data)
task_def = TaskDef.model_validate_json(json_data)
print(task_def)

In [ ]:
agent_def.model_json_schema()

In [ ]:
from agentgenius.config import prompt_lib
from pydantic.dataclasses import dataclass
from pydantic_ai import Tool
import ast
from agentgenius.tools import ToolDef


@dataclass
class ToolCode:
    name: str
    code: str


class CoderTask(TaskDef):
    def __init__(self, question=""):
        self.name = "coder"
        self.question = "write a python function give answer to my question: " + question
        self.priority = 1
        self.agent = AgentDef(
            model="openai:gpt-4o",
            name="coder",
            system_prompt=prompt_lib["tool_agent"],
            params={
                "retries": 2,
                "result_type": ToolCode,
            },
        )
        # self.toolset = ToolSet([])

    def _register(self, tool: ToolCode):
        tree = ast.parse(tool.code)
        function_def = next((node for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)), None)

        function_name = tool.name

        # exec_globals = {"RunContext": RunContext}
        exec_globals = globals()
        exec(tool.code, exec_globals)

        generated_function = exec_globals.get(function_name)

        self.agent.register_tool(Tool(generated_function))

        return f"Tool '{function_name}' successfully generated and registered."


c = CoderTask("what time is it?")
coder = Task(c)
code = coder.run_sync().data
c._register(code)
# print(coder.run_sync().data)

In [ ]:
import ast
import json

from pydantic import TypeAdapter
from pydantic.dataclasses import dataclass
from pydantic_ai import Tool

from agentgenius.config import prompt_lib
from agentgenius.tools import ToolDef
from agentgenius.agents import AgentDef
from agentgenius.tasks import Task, TaskDef
from agentgenius.tools import ToolSet


def get_datetime():
    return "2023-01-01 00:00:00"


class TaskAgent(TaskDef):
    def __init__(self, question=""):
        self.name = "task_agent"
        self.question = question
        self.priority = 1
        self.agent = AgentDef(
            model="openai:gpt-4o",
            name="coder",
            system_prompt="""You are a task planner. you create a task for other agents. 
            Do not answer the user questions. Just make a plan how to do this.""",
            params={"retries": 2, "result_type": Task},
        )


test_task = TaskDef(
    name="get_datetime",
    question="Retrieve the current date and time from the system.",
    priority=1,
    agent=AgentDef(
        model="openai:gpt-4o", name="TimeFetcher", system_prompt="You fetch the current date and time from the system."
    ),
    toolset=ToolSet([get_datetime]),
)


# def model_dump_json(cls, instance: object):
#     adapter = TypeAdapter(cls)
#     return adapter.dump_json(instance, indent=2, by_alias=True, exclude_unset=True, serialize_as_any=True)


task_agent = Task(TaskAgent(test_task))


print(task_agent.run_sync().data)
# json.dumps(asdict(task_agent))

In [ ]:
print(TypeAdapter(Task).validate_json(task_agent))

In [10]:
from pydantic import TypeAdapter
from pydantic.dataclasses import dataclass

In [ ]:
from agentgenius.agents import AgentDef
from typing import Type


a = AgentDef(
    model="openai:gpt-4o",
    name="coder",
    system_prompt="You are a task planner. you create a task for other agents. Do not answer the user questions. Just make a plan how to do this.",
    params={"retries": 2, "result_type": str},
)
print(a)
print(a.__dict__)
AgentDef.model_dump_json(a)